

```
Citation:- https://www.youtube.com/watch?v=ba42uYJd8nc,
https://colab.research.google.com/drive/1XHNNYwDYYoaJaQVCy1uIeKqErXTYpwLu?usp=sharing
```



In [1]:
!wget https://bitbucket.org/ishaanjav/code-and-deploy-custom-tensorflow-lite-model/raw/a4febbfee178324b2083e322cdead7465d6fdf95/fruits.zip

--2024-04-18 20:54:12--  https://bitbucket.org/ishaanjav/code-and-deploy-custom-tensorflow-lite-model/raw/a4febbfee178324b2083e322cdead7465d6fdf95/fruits.zip
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c3:9b0a, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105946856 (101M) [application/zip]
Saving to: ‘fruits.zip’

fruits.zip          100%[===================>] 101.04M  21.5MB/s    in 5.9s    

2024-04-18 20:54:20 (17.3 MB/s) - ‘fruits.zip’ saved [105946856/105946856]



In [2]:
!unzip fruits.zip

Archive:  fruits.zip
   creating: fruits/
  inflating: __MACOSX/._fruits       
  inflating: fruits/.DS_Store        
  inflating: __MACOSX/fruits/._.DS_Store  
   creating: fruits/test/
   creating: fruits/train/
   creating: fruits/validation/
   creating: fruits/test/apple/
  inflating: fruits/test/.DS_Store   
  inflating: __MACOSX/fruits/test/._.DS_Store  
   creating: fruits/test/banana/
   creating: fruits/test/orange/
   creating: fruits/train/apple/
  inflating: fruits/train/.DS_Store  
  inflating: __MACOSX/fruits/train/._.DS_Store  
   creating: fruits/train/banana/
   creating: fruits/train/orange/
   creating: fruits/validation/apple/
  inflating: fruits/validation/.DS_Store  
  inflating: __MACOSX/fruits/validation/._.DS_Store  
   creating: fruits/validation/banana/
   creating: fruits/validation/orange/
  inflating: fruits/test/apple/Screen Shot 2018-06-08 at 5.27.19 PM.png  
  inflating: __MACOSX/fruits/test/apple/._Screen Shot 2018-06-08 at 5.27.19 PM.png  
  inflatin

In [3]:
import tensorflow as tf

In [4]:
trainingDataSet = tf.keras.utils.image_dataset_from_directory(
    "fruits/train",
    image_size = (32, 32),
    batch_size = 20
)
validationDataSet = tf.keras.utils.image_dataset_from_directory(
    "fruits/validation",
    image_size = (32, 32),
    batch_size = 20
)
testingDataSet = tf.keras.utils.image_dataset_from_directory(
    "fruits/test",
    image_size = (32, 32),
    batch_size = 20
)

Found 460 files belonging to 3 classes.
Found 66 files belonging to 3 classes.
Found 130 files belonging to 3 classes.


In [5]:
image_classification_model = tf.keras.Sequential(
    [
     tf.keras.layers.Rescaling(1./255),  # layer for rescaling the size of the image between 0 to 1 instead of 0 to 255
     tf.keras.layers.Conv2D(32, 3, activation="relu"), # convolution layer to filter the images by using 32 filters
     tf.keras.layers.MaxPooling2D(),   # maxpooling is used to get important features from the image into the form of 2x2 matrix
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation="relu"), # dense layer is used to build neural network with 128 neurons
     tf.keras.layers.Dense(3) # last layer with 3 neurons because to get what model thinks about each class in dataset that are oranges, apples and bananas
    ]
)

In [6]:
# compiling model
image_classification_model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [7]:
# model training on training dataset and while training validating the model using validation dataset
image_classification_model.fit(
    trainingDataSet,
    validation_data = validationDataSet,
    epochs = 10
)

Epoch 1/10
23/23 [==============================] - 6s 60ms/step - loss: 0.9955 - accuracy: 0.5587 - val_loss: 0.8092 - val_accuracy: 0.5455
Epoch 2/10
23/23 [==============================] - 2s 50ms/step - loss: 0.5978 - accuracy: 0.7457 - val_loss: 0.7131 - val_accuracy: 0.6970
Epoch 3/10
23/23 [==============================] - 2s 48ms/step - loss: 0.4338 - accuracy: 0.8217 - val_loss: 0.3544 - val_accuracy: 0.8636
Epoch 4/10
23/23 [==============================] - 2s 68ms/step - loss: 0.2854 - accuracy: 0.8826 - val_loss: 0.3312 - val_accuracy: 0.8939
Epoch 5/10
23/23 [==============================] - 2s 59ms/step - loss: 0.2415 - accuracy: 0.9022 - val_loss: 0.2316 - val_accuracy: 0.8939
Epoch 6/10
23/23 [==============================] - 2s 48ms/step - loss: 0.1809 - accuracy: 0.9174 - val_loss: 0.3577 - val_accuracy: 0.8788
Epoch 7/10
23/23 [==============================] - 2s 49ms/step - loss: 0.1476 - accuracy: 0.9522 - val_loss: 0.5181 - val_accuracy: 0.8182
Epoch 8/10
23

In [9]:
# testing the model using testing dataset
image_classification_model.evaluate(testingDataSet)

7/7 [==============================] - 0s 3ms/step - loss: 0.1435 - accuracy: 0.9538


[0.14346209168434143, 0.9538461565971375]

In [10]:
# converting model into TensorFlow lite model so that we can use this model on android device to classify images
converter = tf.lite.TFLiteConverter.from_keras_model(image_classification_model)
tflite_image_classification_model = converter.convert()

with open("image_classification_model.tflite", 'wb') as f:
  f.write(tflite_image_classification_model)